In [1]:
# Instalação das bibliotecas (execute apenas se não estiverem instaladas)
# !pip install gensim matplotlib scikit-learn pandas numpy spacy plotly

# Importações básicas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
import gensim.downloader as api
from sklearn.manifold import TSNE
import re
import nltk
from nltk.tokenize import word_tokenize
import warnings
warnings.filterwarnings('ignore')

# Download de recursos NLTK (se necessário)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
# Dados de exemplo - críticas de filmes (simplificadas)
textos = [
    "Este filme é incrível, adorei a atuação do protagonista",
    "A direção de fotografia é espetacular e o roteiro é envolvente",
    "Péssimo filme, desperdicei meu tempo assistindo isso",
    "Os atores são talentosos mas o roteiro é fraco",
    "Cinematografia belíssima, recomendo assistir no cinema",
    "Não gostei da história, personagens mal desenvolvidos",
    "A trilha sonora combina perfeitamente com as cenas",
    "Filme entediante, previsível do início ao fim",
    "Os efeitos especiais são impressionantes, tecnologia de ponta",
    "História emocionante, chorei no final do filme"
]

# Verificando os dados
for i, texto in enumerate(textos[:3]):  # Mostrando apenas os 3 primeiros
    print(f"Texto {i+1}: {texto}")

Texto 1: Este filme é incrível, adorei a atuação do protagonista
Texto 2: A direção de fotografia é espetacular e o roteiro é envolvente
Texto 3: Péssimo filme, desperdicei meu tempo assistindo isso


In [10]:
from nltk.corpus import stopwords

def preprocessar_texto(texto):
    # Converter para minúsculas
    texto = texto.lower()

    # Remover caracteres especiais e números
    texto = re.sub(r'[^a-záàâãéèêíïóôõöúçñ ]', '', texto)

    # Tokenizar
    tokens = word_tokenize(texto)

    # Remover stopwords (opcional, dependendo da aplicação)
    stop_words = set(stopwords.words('portuguese'))
    tokens = [token for token in tokens if token not in stop_words]

    return tokens

# Aplicar pré-processamento a todos os textos
textos_preprocessados = [preprocessar_texto(texto) for texto in textos]

# Verificar resultado
print("Exemplo de texto original:")
print(textos[9])
print("\nDepois do pré-processamento:")
print(textos_preprocessados[9])

Exemplo de texto original:
História emocionante, chorei no final do filme

Depois do pré-processamento:
['história', 'emocionante', 'chorei', 'final', 'filme']


In [7]:
# Definir parâmetros do modelo
vector_size = 100    # Dimensionalidade dos vetores
window = 5           # Tamanho da janela de contexto
min_count = 1        # Frequência mínima das palavras
workers = 4          # Número de threads para treinamento
sg = 1               # Modelo Skip-gram (1) ou CBOW (0)

# Treinar o modelo
model = Word2Vec(
    sentences=textos_preprocessados,
    vector_size=vector_size,
    window=window,
    min_count=min_count,
    workers=workers,
    sg=sg
)

print(f"Modelo treinado com {len(model.wv.key_to_index)} palavras no vocabulário")


Modelo treinado com 44 palavras no vocabulário


In [9]:
# Listar algumas palavras do vocabulário
palavras = list(model.wv.key_to_index.keys())
print("Algumas palavras do vocabulário:")
print(palavras[:10])  # Primeiras 10 palavras

# Verificar o vetor de uma palavra específica
if 'filme' in model.wv:
    vetor_filme = model.wv['filme']
    print(f"\nVetor da palavra 'filme' (primeiras 10 dimensões):")
    print(vetor_filme[:10])
    print(f"Dimensionalidade do vetor: {len(vetor_filme)}")


Algumas palavras do vocabulário:
['filme', 'história', 'roteiro', 'desperdicei', 'belíssima', 'cinematografia', 'fraco', 'talentosos', 'atores', 'assistindo']

Vetor da palavra 'filme' (primeiras 10 dimensões):
[-0.00053898  0.00023755  0.00510847  0.00900957 -0.00930602 -0.00711791
  0.00645901  0.00897323 -0.00500892 -0.00376375]
Dimensionalidade do vetor: 100


In [11]:
if 'filme' in model.wv:
    similares = model.wv.most_similar('história', topn=5)
    print("\nPalavras mais similares a 'história':")
    for palavra, similaridade in similares:
        print(f"{palavra}: {similaridade:.4f}")


Palavras mais similares a 'história':
tecnologia: 0.1889
final: 0.1886
espetacular: 0.1610
atuação: 0.1591
envolvente: 0.1372


In [14]:
model.save("palavras.model")
vascao = Word2Vec.load("palavras.model")
print(vascao)

Word2Vec<vocab=44, vector_size=100, alpha=0.025>


In [17]:
print([nome for nome in list(api.info()['models'].keys())[:10]])

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50']


In [18]:
modelo_pretreinado = api.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


In [20]:
if 'computer' in modelo_pretreinado:
        similares = modelo_pretreinado.most_similar('computer', topn=5)
        print("\nPalavras mais similares a 'computer':")
        for palavra, similaridade in similares:
            print(f"{palavra}: {similaridade:.4f}")


Palavras mais similares a 'computer':
computers: 0.8752
software: 0.8373
technology: 0.7642
pc: 0.7366
hardware: 0.7290


In [23]:
def calcular_similaridade(modelo, pares_palavras):
    resultados = []
    for par in pares_palavras:
        palavra1, palavra2 = par
        if palavra1 in modelo.wv and palavra2 in modelo.wv:
            similaridade = modelo.wv.similarity(palavra1, palavra2)
            resultados.append((par, similaridade))
        else:
            resultados.append((par, "Uma ou ambas as palavras não estão no vocabulário"))
    return resultados
pares = [
    ('filme', 'cinema'),
    ('bom', 'ruim'),
    ('ator', 'atuação'),
    ('filme', 'protagonista')
]

# Calcular similaridades
similaridades = calcular_similaridade(model, pares)
print("\nSimilaridade entre pares de palavras:")
for (palavra1, palavra2), similaridade in similaridades:
    if isinstance(similaridade, float):
        print(f"{palavra1} - {palavra2}: {similaridade:.4f}")
    else:
        print(f"{palavra1} - {palavra2}: {similaridade}")


Similaridade entre pares de palavras:
filme - cinema: 0.1693534404039383
bom - ruim: Uma ou ambas as palavras não estão no vocabulário
ator - atuação: Uma ou ambas as palavras não estão no vocabulário
filme - protagonista: -0.11359719187021255


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

# Dados rotulados para exemplo
textos_rotulados = textos  # Usando os mesmos textos de antes
sentimentos = [1, 1, 0, 0, 1, 0, 1, 0, 1, 1]  # 1: positivo, 0: negativo

# Função para gerar vetores de documento usando embeddings
def texto_para_vetor(texto, modelo):
    """Converte um texto em um vetor médio dos embeddings de suas palavras"""
    palavras = preprocessar_texto(texto)
    # Filtrar palavras que estão no vocabulário do modelo
    palavras_no_vocab = [p for p in palavras if p in modelo.wv]
    if not palavras_no_vocab:
        # Se nenhuma palavra estiver no vocabulário, retorna vetor de zeros
        return np.zeros(modelo.vector_size)
    # Calcular a média dos vetores das palavras
    vetores = [modelo.wv[palavra] for palavra in palavras_no_vocab]
    return np.mean(vetores, axis=0)

# Dividir dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    textos_rotulados, sentimentos, test_size=0.3, random_state=42
)

# 1. Abordagem com TF-IDF
vectorizer = TfidfVectorizer(max_features=100)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

clf_tfidf = LogisticRegression(random_state=42)
clf_tfidf.fit(X_train_tfidf, y_train)
y_pred_tfidf = clf_tfidf.predict(X_test_tfidf)

# 2. Abordagem com Word Embeddings
X_train_emb = np.array([texto_para_vetor(texto, model) for texto in X_train])
X_test_emb = np.array([texto_para_vetor(texto, model) for texto in X_test])

clf_emb = LogisticRegression(random_state=42)
clf_emb.fit(X_train_emb, y_train)
y_pred_emb = clf_emb.predict(X_test_emb)

# Comparar resultados
print("\nResultados com TF-IDF:")
print(classification_report(y_test, y_pred_tfidf))

print("\nResultados com Word Embeddings:")
print(classification_report(y_test, y_pred_emb))
